# Asynchronous GraphQL Data Fetching data Mahasiswa
Notebook ini menunjukkan bagaimana kita bisa melakukan asynchronous data fetching dengan GraphQL SiCekCok untuk mengambil data mahasiswa menggunakan Python. Notebook ini menggunakan library `aiohttp` untuk melakukan HTTP request secara asynchronous dan `aiographql` untuk melakukan query ke GraphQL server.

### Langkah 0: Install Pustaka yang Diperlukan
Pada langkah Persiapan, kita menginstall pustaka `aiohttp` untuk melakukan HTTP request secara asynchronous dan `pandas` untuk memproses data dalam format DataFrame.

In [1]:
%pip install aiohttp pandas

Note: you may need to restart the kernel to use updated packages.


### Langkah 1: Mengimpor Pustaka yang Diperlukan
Pada langkah pertama, kita mengimpor pustaka `aiohttp` `asyncio`  untuk untuk melakukan HTTP request secara asynchronous dan `pandas` untuk memproses data dalam format DataFrame.


In [2]:
import aiohttp
import asyncio
import pandas as pd

### Langkah 2: Mendefinisikan Fungsi untuk Query GraphQL
Kita mendefinisikan fungsi `query_graphql` yang mengambil parameter `nim` dan mengirimkan permintaan POST ke endpoint GraphQL yang ditentukan. Fungsi ini mengembalikan hasil dalam format JSON.

In [3]:
async def query_graphql(session, nim):
    url = 'https://sicekcok.if.unismuh.ac.id/graphql'  # Ganti dengan URL endpoint GraphQL yang sesuai
    query = """
    query($nim: String!) {
      mahasiswa(nim: $nim) {
        nim
        kodeProdi
        angkatan
        nama
        jenisKelamin
        semesterAwal
        tahunAkademikLulus
        tanggalLulus
        lulus
        masaStudi
        khs {
          tahunAkademik
          ips
          sksSmt
          ipk
          sksTotal
          statusMahasiswa
        }
      }
    }
    """
    variables = {'nim': str(nim)}  # Pastikan NIM dikonversi ke string
    async with session.post(url, json={'query': query, 'variables': variables}) as response:
        return await response.json()

### Langkah 3: Membaca File NIM dan Menyiapkan Daftar Data
Langkah ini membaca file `nim.csv` yang berisi daftar NIM dan menyiapkan struktur data untuk menyimpan hasil unduhan.

In [4]:
nim_list = pd.read_csv('../data/nim.csv')['nim'].tolist()
data_list = []
total = len(nim_list)
max_khs = 0
print(f"Total NIM: {total}")

Total NIM: 13


### Langkah 4: Mengunduh Data secara asynchronousdan Memprosesnya 
Kita mengiterasi daftar NIM, mengunduh data untuk setiap NIM menggunakan fungsi `query_graphql`, dan memproses data tersebut untuk disimpan dalam daftar `data_list`. Juga, kita melacak jumlah maksimum entri KHS untuk mengatur kolom DataFrame nantinya.

In [5]:
async def get_mahasiswa():
    async with aiohttp.ClientSession() as session:
        tasks = []
        for nim in nim_list:
            tasks.append(query_graphql(session, nim))
        responses = await asyncio.gather(*tasks)
        for i, result in enumerate(responses):
            nim = nim_list[i]
            # print(f"Result for NIM {nim}: {result.get('data', {}).get('mahasiswa', {}).get('nama')}")
            mahasiswa = result.get('data', {}).get('mahasiswa', {})
            if mahasiswa:
                row = {
                    'kodeProdi': mahasiswa.get('kodeProdi'),
                    'angkatan': mahasiswa.get('angkatan'),
                    'semesterAwal': mahasiswa.get('semesterAwal'),
                    'nim': mahasiswa.get('nim'),
                    'nama': mahasiswa.get('nama'),
                    'jenisKelamin': mahasiswa.get('jenisKelamin'),
                    'tahunAkademikLulus': mahasiswa.get('tahunAkademikLulus'),
                    'tanggalLulus': mahasiswa.get('tanggalLulus'),
                    'lulus': mahasiswa.get('lulus'),
                    'masaStudi': mahasiswa.get('masaStudi'),
                }
                khs = mahasiswa.get('khs', [])
                global max_khs
                max_khs = max(max_khs, len(khs))
                for idx, k in enumerate(khs, start=1):
                    row.update({
                        f'khs{idx}_tahunAkademik': k.get('tahunAkademik'),
                        f'khs{idx}_ips': k.get('ips'),
                        f'khs{idx}_sksSmt': k.get('sksSmt'),
                        f'khs{idx}_ipk': k.get('ipk'),
                        f'khs{idx}_sksTotal': k.get('sksTotal'),
                        f'khs{idx}_statusMahasiswa': k.get('statusMahasiswa'),
                    })
                data_list.append(row)
                # print(f'Downloading data {i + 1}/{total} berhasil di download')
await get_mahasiswa()
print(f'{total} Data berhasil di download')

13 Data berhasil di download


### Langkah 5: Membuat DataFrame dan Menyimpannya ke CSV
Kita membuat DataFrame dari daftar data yang telah diproses.

In [6]:
df = pd.DataFrame(data_list)
columns = [
    'kodeProdi', 'angkatan', 'semesterAwal', 'nim', 'nama', 'jenisKelamin', 'tahunAkademikLulus', 'tanggalLulus', 'lulus', 'masaStudi'
] + [
    f'khs{idx}_{field}' for idx in range(1, max_khs + 1) for field in
    ['tahunAkademik', 'ips', 'sksSmt', 'ipk', 'sksTotal', 'statusMahasiswa']
]
df = df.reindex(columns=columns)


### Langkah 6: Memverifikasi Data
Langkah terakhir adalah memverifikasi beberapa baris pertama dari DataFrame untuk memastikan bahwa data telah diunduh dan diproses dengan benar.

In [7]:
pd.set_option('display.max_columns', None)  # Menampilkan semua kolom
pd.set_option('display.expand_frame_repr', False)  # Menghindari pembungkusan frame
df.head(10)

,kodeProdi,angkatan,semesterAwal,nim,nama,jenisKelamin,tahunAkademikLulus,tanggalLulus,lulus,masaStudi,khs1_tahunAkademik,khs1_ips,khs1_sksSmt,khs1_ipk,khs1_sksTotal,khs1_statusMahasiswa,khs2_tahunAkademik,khs2_ips,khs2_sksSmt,khs2_ipk,khs2_sksTotal,khs2_statusMahasiswa,khs3_tahunAkademik,khs3_ips,khs3_sksSmt,khs3_ipk,khs3_sksTotal,khs3_statusMahasiswa,khs4_tahunAkademik,khs4_ips,khs4_sksSmt,khs4_ipk,khs4_sksTotal,khs4_statusMahasiswa,khs5_tahunAkademik,khs5_ips,khs5_sksSmt,khs5_ipk,khs5_sksTotal,khs5_statusMahasiswa,khs6_tahunAkademik,khs6_ips,khs6_sksSmt,khs6_ipk,khs6_sksTotal,khs6_statusMahasiswa,khs7_tahunAkademik,khs7_ips,khs7_sksSmt,khs7_ipk,khs7_sksTotal,khs7_statusMahasiswa,khs8_tahunAkademik,khs8_ips,khs8_sksSmt,khs8_ipk,khs8_sksTotal,khs8_statusMahasiswa,khs9_tahunAkademik,khs9_ips,khs9_sksSmt,khs9_ipk,khs9_sksTotal,khs9_statusMahasiswa,khs10_tahunAkademik,khs10_ips,khs10_sksSmt,khs10_ipk,khs10_sksTotal,khs10_statusMahasiswa,khs11_tahunAkademik,khs11_ips,khs11_sksSmt,khs11_ipk,khs11_sksTotal,khs11_statusMahasiswa,khs12_tahunAkademik,khs12_ips,khs12_sksSmt,khs12_ipk,khs12_sksTotal,khs12_statusMahasiswa,khs13_tahunAkademik,khs13_ips,khs13_sksSmt,khs13_ipk,khs13_sksTotal,khs13_statusMahasiswa,khs14_tahunAkademik,khs14_ips,khs14_sksSmt,khs14_ipk,khs14_sksTotal,khs14_statusMahasiswa,khs15_tahunAkademik,khs15_ips,khs15_sksSmt,khs15_ipk,khs15_sksTotal,khs15_statusMahasiswa,khs16_tahunAkademik,khs16_ips,khs16_sksSmt,khs16_ipk,khs16_sksTotal,khs16_statusMahasiswa
0,20201,2017,20171,105821100817,ASWIN DWI PUTRA,L,20222,2023-08-30,True,"5 Tahun, 11 Bulan",20171,3.36,22,3.36,22,A,20172,3.19,21,3.28,43,A,20181,1.14,22,3.07,54,A,20182,1.24,17,2.97,63,A,20191,0.59,17.0,2.94,67.0,A,20192,3.29,14.0,3.00,81.0,A,20201,0.83,24.0,2.96,89.0,A,20202,1.57,14.0,2.94,97.0,A,20211,3.25,16.0,2.98,113.0,A,20212,2.55,24.0,2.97,134.0,A,20213,2.81,8.0,3.08,134.0,A,20221,2.85,17.0,3.14,147.0,A,20222,3.75,15.0,3.24,156.0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20201,2017,20171,105821100917,MUHAMMAD ASRUL,L,20231,2024-01-06,True,"6 Tahun, 4 Bulan",20171,0.00,22,0.00,0,A,20172,0.00,21,0.00,0,A,20181,3.14,14,3.14,14,A,20182,2.79,24,2.92,38,A,20191,2.80,20.0,2.98,56.0,A,20192,1.70,20.0,2.96,68.0,A,20201,2.94,17.0,2.95,85.0,A,20202,3.00,20.0,3.02,103.0,A,20203,3.00,3.0,3.02,106.0,A,20211,3.68,22.0,3.13,128.0,A,20212,2.91,17.0,3.14,143.0,A,20221,0.00,12.0,3.13,142.0,A,20222,1.33,12.0,3.15,146.0,A,20231,4.00,8.0,3.20,154.0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20201,2017,20171,105821104017,ANDI RAHMAT AL-FARIZIH,L,None,None,False,None,20171,1.18,22,2.89,9,A,20172,1.57,21,2.46,24,A,20181,1.12,17,2.36,33,A,20182,1.94,17,2.31,48,A,20191,2.59,17.0,2.46,63.0,A,20192,2.50,20.0,2.53,81.0,A,20201,2.20,20.0,2.60,95.0,A,20202,0.95,20.0,2.56,104.0,A,20211,1.84,16.0,2.62,113.0,A,20212,0.47,17.0,2.64,115.0,A,20221,0.00,0.0,2.64,115.0,N,20222,0.00,0.0,2.64,115.0,N,20231,0.00,0.0,2.64,115.0,N,20232,0.00,0.0,2.64,114.0,N,20241,0.00,0.0,2.64,115.0,N,NaN,NaN,NaN,NaN,NaN,NaN
3,20201,2017,20171,105821111017,UCOK AKMAL,L,None,None,False,None,20171,0.14,22,1.00,3,A,20172,0.00,21,1.00,3,A,20182,0.00,0,1.00,3,N,20231,0.00,0,1.00,3,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20201,2017,20171,105821101117,SRI REZKIYANTI,P,None,None,False,None,20171,3.27,22,3.60,20,A,20172,2.86,21,3.22,41,A,20181,0.40,20,3.26,43,A,20182,0.00,0,3.26,43,N,20191,0.00,0.0,3.26,43.0,N,20192,0.00,0.0,3.26,43.0,N,20201,0.00,0.0,3.26,43.0,N,20202,0.00,0.0,3.26,43.0,N,20211,0.00,0.0,3.26,43.0,N,20212,0.00,0.0,3.26,43.0,N,20221,0.00,0.0,3.26,43.0,N,20222,0.00,0.0,3.26,43.0,N,20231,0.00,0.0,3.26,43.0,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,20201,2017,20171,105821101017,ARIFUDDIN,L,20

# Validasi data yang sudah di download dengan NIM yang dicari

In [8]:
# Validasi data yang sudah di download dengan NIM yang dicari
nim_yang_dicari = '105821100817'  # Ganti dengan NIM yang ingin Anda cari
data_nim = df.loc[df['nim'] == nim_yang_dicari]

# Menampilkan hasil pencarian dan cocokkan dengan KHS yang ada
if not data_nim.empty:
    for index, row in data_nim.iterrows():
        print(f"Data Mahasiswa dengan NIM: {row['nim']}")
        print("=================================")
        for column in data_nim.columns:
            print(f"{column}: {row[column]}")
else:
    print("Data dengan NIM tersebut tidak ditemukan.")

Data Mahasiswa dengan NIM: 105821100817
kodeProdi: 20201
angkatan: 2017
semesterAwal: 20171
nim: 105821100817
nama: ASWIN DWI PUTRA
jenisKelamin: L
tahunAkademikLulus: 20222
tanggalLulus: 2023-08-30
lulus: True
masaStudi: 5 Tahun, 11 Bulan
khs1_tahunAkademik: 20171
khs1_ips: 3.36
khs1_sksSmt: 22
khs1_ipk: 3.36
khs1_sksTotal: 22
khs1_statusMahasiswa: A
khs2_tahunAkademik: 20172
khs2_ips: 3.19
khs2_sksSmt: 21
khs2_ipk: 3.28
khs2_sksTotal: 43
khs2_statusMahasiswa: A
khs3_tahunAkademik: 20181
khs3_ips: 1.14
khs3_sksSmt: 22
khs3_ipk: 3.07
khs3_sksTotal: 54
khs3_statusMahasiswa: A
khs4_tahunAkademik: 20182
khs4_ips: 1.24
khs4_sksSmt: 17
khs4_ipk: 2.97
khs4_sksTotal: 63
khs4_statusMahasiswa: A
khs5_tahunAkademik: 20191
khs5_ips: 0.59
khs5_sksSmt: 17.0
khs5_ipk: 2.94
khs5_sksTotal: 67.0
khs5_statusMahasiswa: A
khs6_tahunAkademik: 20192
khs6_ips: 3.29
khs6_sksSmt: 14.0
khs6_ipk: 3.0
khs6_sksTotal: 81.0
khs6_statusMahasiswa: A
khs7_tahunAkademik: 20201
khs7_ips: 0.83
khs7_sksSmt: 24.0
khs7_ipk: 

# Simpan Ke File .csv

In [12]:
from datetime import datetime

# Generate the timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Create the filename with the timestamp
filename = f"../outputs/mahasiswa_data_{timestamp}.csv"

# Save the DataFrame to the CSV file
df.to_csv(filename, index=False)

# Print the success message with the filename
print(f"Data berhasil disimpan ke file {filename}")

Data berhasil disimpan ke file ../outputs/mahasiswa_data_20250118_142044.csv
